In [1]:
from itertools import combinations
import pandas as pd

In [2]:
dataset = pd.read_csv('../data/transactions.csv')

In [3]:
class Aprio:

    def __init__(self, dataset, min_support, min_confidence):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.transactions = self.list_from_dataframe(dataset)
        self.item_sets = self.generate_item_sets()
        self.frequent_items = self.get_frequent_itemsets()
        self.items_count = self.get_items_count()
        self.rules = self.generate_rules(self.frequent_items, self.min_confidence)
        
    def get_items_count(self):
        print("====> getting items' count")
        items_count = {}
        for transaction in self.transactions:
            for itemset in self.frequent_items:
                if itemset.issubset(transaction):
                    items_count[itemset] = items_count.get(itemset, 0) + 1
        return items_count

    """
        input: transactions in form of a dataframe
        output: a 2-D list of transactions [[item1, item2...], [item1, item3...]]
    """
    def list_from_dataframe(self, dataset):
        print("====> getting list of transactions")
        list_ = []
        for index in range(dataset.shape[0]):
            row = dataset.iloc[index].dropna()
            list_.append(list(row.values))
        return list_
    
    """
        needed data: 2-D list of transactions
        output: a list of all items in all transactions 
    """
    def generate_item_sets(self):
        print("====> generating item sets")
        return [frozenset([item]) for transaction in self.transactions for item in transaction]
    

    """
        removes items that does not verify the minimum support condition
    """
    def prune_min_supp(self, candidate_counts):
        print("====> pruning min support")
        return {itemset for itemset, count in candidate_counts.items() if count  >= self.min_support}
        

    def prune_subsets(self,candidates, prev_frequent_itemsets):
        print("====> pruning")
        pruned_candidates = set()
        for candidate in candidates:
            is_valid = True
            subsets = combinations(candidate, len(candidate) - 1)
            for subset in subsets:
                if frozenset(subset) not in prev_frequent_itemsets:
                    is_valid = False
                    break
            if is_valid:
                pruned_candidates.add(candidate)
        return pruned_candidates


    def generate_next_candidates_set(self, prev_candidates, k):
        print("====> generating next condidates")
        candidates = set()
        for itemset1 in prev_candidates:
            for itemset2 in prev_candidates:
                union_set = itemset1.union(itemset2)
                if len(union_set) == k:
                    candidates.add(union_set)
        return candidates


    """
        needed data: 2-D list of transactions
        output: list of union(F_k), frequent items in eatch iteration 
    """
    def get_frequent_itemsets(self):
        print("====> getting frequent items")
        itemsets = self.item_sets.copy()
        frequent_itemsets = []
        
        k = 2

        while itemsets:

            candidate_counts = {}
            for transaction in self.transactions:
                for candidate in itemsets:
                    if candidate.issubset(transaction):
                        candidate_counts[candidate] = candidate_counts.get(candidate, 0) + 1
            
            frequent_itemsets_k = self.prune_min_supp(candidate_counts)
            
            candidates_k = self.generate_next_candidates_set(frequent_itemsets_k, k)
            
            candidates_k = self.prune_subsets(candidates_k, frequent_itemsets_k)
            
            frequent_itemsets.extend(frequent_itemsets_k)

            itemsets = candidates_k
            
            k += 1
        
        return frequent_itemsets

    def calculate_confidence(self, itemset, antecedent):
        return self.items_count[itemset] / self.items_count[antecedent]

    def calculate_lift(self, confidence, consequent):
        return confidence / self.calculate_support(consequent)

    def calculate_support(self, itemset):
        return self.items_count[itemset] / len(self.transactions)

   
    def generate_rules(self, frequent_itemsets, min_confidence):
        print("====> generating rulesl")
        rules = []
        for itemset in frequent_itemsets:
            if len(itemset) > 1:
                itemset_list = list(itemset)
                for i in range(1, len(itemset)):
                    antecedent = frozenset(itemset_list[:i])
                    consequent = frozenset(itemset_list[i:])
                    confidence = self.calculate_confidence(itemset, antecedent)
                    lift = self.calculate_lift(confidence, consequent)
                    if confidence >= min_confidence:
                        if len(list(antecedent)) > 0 and len(list(consequent)) > 0:
                            rules.append({"antecedent": list(antecedent), "consequent": list(consequent), "confidence": confidence, "lift": lift})
        return rules
    

In [10]:
apr = Aprio(dataset.head(1000), 4, 0.3)      

====> getting list of transactions
====> generating item sets
====> getting frequent items
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> pruning min support
====> generating next condidates
====> pruning
====> getting items' count
====> generating rulesl


In [8]:
len(apr.rules)

2527

In [9]:
rules_df = pd.DataFrame(apr.rules)
rules_df.shape

rules_df.to_csv('../results/s4_c03_t5_m30.csv')

In [44]:
import json 
import re

def rules_to_json(rules, path):   

    def filter_fields(dictionary, fields_to_keep):
        return {key: value for key, value in dictionary.items() if key in fields_to_keep}

    filtered_data = [filter_fields(item, ['antecedent', 'consequent', 'lift', 'confidence']) for item in rules]

    json_data = json.dumps(filtered_data, indent=2)
    with open(f'{path}/rules.json', 'w') as json_file:
        json_file.write(json_data)
        json_file.close()
        print("successfully wrote results.")



def rules_to_markdown(rules, path):
    rules_file = open(f'{path}/rules.md', 'w')
    line = f"<ol>\n"
    for rule in rules:
        line += f"<li>"
        
        line += f"{rule['antecedent']}"
        
        line += f" ========> "
        
        line += f"{rule['consequent']} "
        line += f"</li>\n"
    line += f"</ol>"
    rules_file.write(line)

    rules_file.close()

def rules_to_xlsx(rules):
    pd.DataFrame(rules).to_csv('../results/results.xlsx')

def map_rules(rules):
    mapping = open("../data/map_stockCode_item.json", "r")
    json_map = json.load(mapping)
    
    cp_rules = rules.copy()
    mapped_rules = []
    for rule in cp_rules:
        ant = []
        for item in rule['antecedent']:
            ant.append(json_map[item])

        cons = []
        for item in rule['consequent']:
            cons.append(json_map[item])

        obj = {'antecedent': ', '.join(ant), 'consequent': ', '.join(cons), 'confidence': rule['confidence'], 'lift': rule['lift']}
        mapped_rules.append(obj)

    return mapped_rules


print("mapping to items...")
rules = map_rules(apr.rules)
print("writing to file...")
rules_to_markdown(rules, "../results")
rules_to_json(rules, "../results")
rules_to_xlsx(rules)
rules

mapping to items...
writing to file...
successfully wrote results.


[{'antecedent': 'RED RETROSPOT PLATE',
  'consequent': 'BLUE POLKADOT PLATE',
  'confidence': 0.8571428571428571,
  'lift': 28.57142857142857},
 {'antecedent': 'LUNCH BAG RED RETROSPOT',
  'consequent': 'ROUND SNACK BOXES SET OF4 WOODLAND',
  'confidence': 0.5,
  'lift': 3.0303030303030303},
 {'antecedent': 'LUNCH BAG PINK POLKADOT',
  'consequent': 'LUNCH BAG WOODLAND',
  'confidence': 0.625,
  'lift': 8.928571428571427},
 {'antecedent': 'MINI JIGSAW SPACEBOY',
  'consequent': 'MINI JIGSAW DOLLY GIRL',
  'confidence': 0.7,
  'lift': 17.5},
 {'antecedent': 'RED RETROSPOT BOWL',
  'consequent': 'BLUE POLKADOT PLATE',
  'confidence': 0.5,
  'lift': 16.666666666666668},
 {'antecedent': '60 TEATIME FAIRY CAKE CASES',
  'consequent': 'PACK OF 60 MUSHROOM CAKE CASES',
  'confidence': 0.6,
  'lift': 9.23076923076923},
 {'antecedent': 'ROUND CONTAINER SET OF 5 RETROSPOT',
  'consequent': 'ROUND SNACK BOXES SET OF4 WOODLAND',
  'confidence': 1.0,
  'lift': 6.0606060606060606},
 {'antecedent': '

In [45]:
pd.DataFrame(rules).isna().sum()

antecedent    0
consequent    0
confidence    0
lift          0
dtype: int64